In [1]:
import numpy as np
import pandas as pd

In [2]:
stores = pd.read_csv('stores.csv')
def v(x):
    d = {'A': 1, 'B': 2, 'C': 3}
    if x in d.keys():
        return d[x]
    else:
        return x
stores = stores.applymap(v)
stores.head()

,Store,Type,Size
0,1,1,151315
1,2,1,202307
2,3,2,37392
3,4,1,205863
4,5,2,34875


In [3]:
train = pd.read_csv('train.csv')
train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [4]:
test = pd.read_csv('test.csv')
test.head()

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False


In [5]:
features = pd.read_csv('features.csv')
features.drop(['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'], axis=1, inplace=True)
features.fillna(method='ffill', inplace=True)
features.head()

,Store,Date,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,211.350143,8.106,False


In [6]:
# 训练集
train_s = pd.merge(train, stores, how='left', on='Store')
len(train_s.Weekly_Sales.unique())

359464

In [7]:
train_s_f = pd.merge(train_s, features, how='left', on=['Store', 'Date', 'IsHoliday'])
def b2i(x):
    d = {False: 0, True: 1}
    if x in d.keys():
        return d[x]
    else:
        return x
train_s_f.IsHoliday = train_s_f.IsHoliday.map(b2i)
train_s_f.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,CPI,Unemployment
0,1,1,2010-02-05,24924.50,0,1,151315,42.31,2.572,211.096358,8.106
1,1,1,2010-02-12,46039.49,1,1,151315,38.51,2.548,211.242170,8.106
2,1,1,2010-02-19,41595.55,0,1,151315,39.93,2.514,211.289143,8.106
3,1,1,2010-02-26,19403.54,0,1,151315,46.63,2.561,211.319643,8.106
4,1,1,2010-03-05,21827.90,0,1,151315,46.50,2.625,211.350143,8.106


In [8]:
train_s_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421570 entries, 0 to 421569
Data columns (total 11 columns):
Store           421570 non-null int64
Dept            421570 non-null int64
Date            421570 non-null object
Weekly_Sales    421570 non-null float64
IsHoliday       421570 non-null int64
Type            421570 non-null int64
Size            421570 non-null int64
Temperature     421570 non-null float64
Fuel_Price      421570 non-null float64
CPI             421570 non-null float64
Unemployment    421570 non-null float64
dtypes: float64(5), int64(5), object(1)
memory usage: 38.6+ MB


In [9]:
feature = ['IsHoliday', 'Type', 'Size', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
X_data = train_s_f[feature]
y_data = train_s_f['Weekly_Sales']
print(X_data.shape)
print(y_data.shape)

(421570, 7)
(421570,)


In [10]:
# 测试集
test = pd.read_csv('test.csv')
test_s = pd.merge(test, stores, how='left', on='Store')
test_s_f = pd.merge(test_s, features, how='left', on=['Store', 'Date', 'IsHoliday'])
def b2i(x):
    d = {False: 0, True: 1}
    if x in d.keys():
        return d[x]
    else:
        return x
test_s_f.IsHoliday = test_s_f.IsHoliday.map(b2i)
test_s_f.head()

,Store,Dept,Date,IsHoliday,Type,Size,Temperature,Fuel_Price,CPI,Unemployment
0,1,1,2012-11-02,0,1,151315,55.32,3.386,223.462779,6.573
1,1,1,2012-11-09,0,1,151315,61.24,3.314,223.481307,6.573
2,1,1,2012-11-16,0,1,151315,52.92,3.252,223.512911,6.573
3,1,1,2012-11-23,1,1,151315,56.23,3.211,223.561947,6.573
4,1,1,2012-11-30,0,1,151315,52.34,3.207,223.610984,6.573


In [11]:
samples = pd.read_csv('sampleSubmission.csv')
samples.head()

,Id,Weekly_Sales
0,1_1_2012-11-02,0
1,1_1_2012-11-09,0
2,1_1_2012-11-16,0
3,1_1_2012-11-23,0
4,1_1_2012-11-30,0


# Model

In [14]:
import xgboost as xgb

/home/moonlight/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_data.values, y_data.values, test_size=0.4, random_state=12345)

In [17]:
data_train = xgb.DMatrix(X_train, label=y_train)
data_test = xgb.DMatrix(X_test, label=y_test)
watch_list = [(data_train, 'eval'), (data_test, 'train')]

In [18]:
param = {'max_depth': 10, 'eta': 1, 'silent': 0, 'objective':'reg:linear', 'lambda':0.2, 'eval_metric':'logloss'}
num_rounds = 100
plst = param.items()

In [19]:
linear = xgb.train(plst, data_train, num_boost_round=num_rounds, evals=watch_list)
y_hat = linear.predict(data_test)
result = np.abs(y_hat - y_test.reshape(1,-1))
print('result: {}'.format(result))
print('error: {}'.format(float(np.sum(result))/len(y_hat)))
print(y_hat)
print(y_test)

[0]	eval-logloss:-588119	train-logloss:-589658
[1]	eval-logloss:-588119	train-logloss:-589658
[2]	eval-logloss:-588119	train-logloss:-589658
[3]	eval-logloss:-588119	train-logloss:-589658
[4]	eval-logloss:-588119	train-logloss:-589658
[5]	eval-logloss:-588119	train-logloss:-589658
[6]	eval-logloss:-588119	train-logloss:-589658
[7]	eval-logloss:-588119	train-logloss:-589658
[8]	eval-logloss:-588119	train-logloss:-589658
[9]	eval-logloss:-588119	train-logloss:-589658
[10]	eval-logloss:-588119	train-logloss:-589658
[11]	eval-logloss:-588119	train-logloss:-589658
[12]	eval-logloss:-588119	train-logloss:-589658
[13]	eval-logloss:-588119	train-logloss:-589658
[14]	eval-logloss:-588119	train-logloss:-589658
[15]	eval-logloss:-588119	train-logloss:-589658
[16]	eval-logloss:-588119	train-logloss:-589658
[17]	eval-logloss:-588119	train-logloss:-589658
[18]	eval-logloss:-588119	train-logloss:-589658
[19]	eval-logloss:-588119	train-logloss:-589658
[20]	eval-logloss:-588119	train-logloss:-589658
[2